In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import joblib
from IPython.display import clear_output
import matplotlib.pyplot as plt
from torch.autograd import Variable

import math
import tqdm
import torch
import gpytorch
from matplotlib import pyplot as plt

# Make plots inline
%matplotlib inline

import utils

In [43]:
aluminium_inputs = joblib.load('../Vectors/single_f_aluminium_vectors')
aluminium_labels = joblib.load('../Vectors/single_f_aluminium_labels')
aluminium_dates = joblib.load('../Vectors/single_f_aluminium_dates')

In [44]:
train_batches,validation_batches,test_batches = utils.train_test_split(aluminium_inputs,aluminium_labels,aluminium_dates,aluminium_inputs.shape[0],0.8,0.7,0)

In [45]:
train_data,train_labels,train_dates = train_batches[0]
test_data,test_labels,test_dates = test_batches[0]
val_data,val_labels,val_dates = validation_batches[0]

In [46]:
train_data = torch.from_numpy(train_data)
train_labels = torch.from_numpy(train_labels)
test_data = torch.from_numpy(test_data)
test_labels = torch.from_numpy(test_labels)
val_data = torch.from_numpy(val_data)
val_labels = torch.from_numpy(val_labels)
train_data = torch.squeeze(train_data, dim = 1)
test_data = torch.squeeze(test_data, dim = 1)
val_data = torch.squeeze(val_data, dim = 1)

In [47]:
print(train_data.shape)
print(train_labels.shape)
print(test_data.shape)
print(test_labels.shape)
print(val_data.shape)
print(val_labels.shape)

torch.Size([672, 55])
torch.Size([672])
torch.Size([50, 55])
torch.Size([50])
torch.Size([118, 55])
torch.Size([118])


In [48]:
representation_dim = 2

In [49]:
data_dim = train_data.size(-1)

class LargeFeatureExtractor(torch.nn.Sequential):
    def __init__(self):
        super(LargeFeatureExtractor, self).__init__()
        self.add_module('linear1', torch.nn.Linear(data_dim, 100))
        self.add_module('relu1', torch.nn.ReLU())
        self.add_module('linear2', torch.nn.Linear(100, 20))
        self.add_module('relu3', torch.nn.ReLU())
        self.add_module('linear4', torch.nn.Linear(20, representation_dim))

feature_extractor = LargeFeatureExtractor()

In [50]:
class GPRegressionModel(gpytorch.models.ExactGP):
        def __init__(self, train_x, train_y, likelihood):
            super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
            self.mean_module = gpytorch.means.ConstantMean()
            self.covar_module = gpytorch.kernels.GridInterpolationKernel(
                gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=representation_dim)),
                num_dims=representation_dim, grid_size=100
            )
            '''self.mean_module = gpytorch.means.ConstantMean()
            self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())'''
            self.feature_extractor = feature_extractor

        def forward(self, x):
            # We're first putting our data through a deep net (feature extractor)
            # We're also scaling the features so that they're nice values
            projected_x = self.feature_extractor(x)
            projected_x = projected_x - projected_x.min(0)[0]
            projected_x = 2 * (projected_x / projected_x.max(0)[0]) - 1

            mean_x = self.mean_module(projected_x)
            covar_x = self.covar_module(projected_x)
            return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [51]:
'''noise_prior = gpytorch.priors.GammaPrior(1.1, 0.05)
noise_prior_mode = (noise_prior.concentration - 1) / noise_prior.rate
MIN_INFERRED_NOISE_LEVEL = 1e-5
likelihood = gpytorch.likelihoods.GaussianLikelihood(
    noise_prior=noise_prior,
    noise_constraint=gpytorch.constraints.Interval(
        MIN_INFERRED_NOISE_LEVEL,
        transform=None,
        initial_value=noise_prior_mode,
        upper_bound = 10
    ),
)'''
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = GPRegressionModel(train_data,train_labels, likelihood).double()

if torch.cuda.is_available():
    model = model.cuda()
    likelihood = likelihood.cuda()

In [ ]:

# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam([
    {'params': model.feature_extractor.parameters()},
    {'params': model.covar_module.parameters()},
    {'params': model.mean_module.parameters()},
    {'params': model.likelihood.parameters()},
], lr=0.01)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

training_iterations = 200

training_loss = []
validation_loss = []

def train():
    iterator = tqdm.notebook.tqdm(range(training_iterations))
    for i in iterator:
        # Zero backprop gradients
        optimizer.zero_grad()
        # Get output from model
        output = model(train_data)
        # Calc loss and backprop derivatives
        loss = -mll(output, train_labels)
        training_loss.append(loss.item())
        loss.backward()
        iterator.set_postfix(loss=loss.item())
        optimizer.step()
        
        # Get into evaluation (predictive posterior) mode
        '''model.eval()
        likelihood.eval()
                
        with torch.no_grad(), gpytorch.settings.fast_pred_var(), gpytorch.settings.use_toeplitz(False):
            ## Calculate the validation loss
            val_loss = -mll(model(val_data), val_labels)
            validation_loss.append(val_loss.item())'''
        
    index = [i+1 for i in range(len(training_loss))]
    plt.plot(index, training_loss, label = "train loss")
    #plt.plot(index, validation_loss, label = "val loss")
    plt.xlabel('Iters')
    plt.ylabel('Loss')
    plt.title('Loss')
    plt.legend()
    plt.show()
        

%time train()

  0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:
model.eval()
likelihood.eval()
train_predictions = []

with torch.no_grad(), gpytorch.settings.fast_pred_var():
    train_predictions.append(likelihood(model(train_data)))

In [ ]:
train_pred_means = []
train_pred_uconf = []
train_pred_lconf = []

for p in train_predictions:
    train_pred_means.append(p.mean.numpy())
    l, u = p.confidence_region()
    train_pred_lconf.append(l.detach().numpy())
    train_pred_uconf.append(u.detach().numpy())
    
train_pred_means = train_pred_means[0]
train_pred_lconf = train_pred_lconf[0]
train_pred_uconf = train_pred_uconf[0]

In [ ]:
index = [i+1 for i in range(len(train_pred_means[0]))]

In [ ]:
fig, ax = plt.subplots()
ax.plot(index,train_pred_means,label = 'train predictions')
ax.plot(index,train_labels,label = 'train predictions')
ax.fill_between(index, train_pred_lconf, train_pred_uconf, color='r', alpha=.005)
plt.xlabel('Timesteps')
plt.ylabel('Predictions')
plt.title('Training Learning')
plt.legend()
plt.show()